# 第六部分 元编程 19 动态属性和特性

## 19.1 使用动态属性转换数据

在Python中，数据的属性和处理数据的方法统称属性（attribute）。其实，方法只是可调用的属性。除了这二者之外，我们还可以创建特性（property），在不改变类接口的前提下，使用存取方法（即读值方法和设值方法）修改数据属性。

In [ ]:
"""
osconfeed.py: Script to download the OSCON schedule feed

# BEGIN OSCONFEED_DEMO

    >>> feed = load()  # <1>
    >>> sorted(feed['Schedule'].keys())  # <2>
    ['conferences', 'events', 'speakers', 'venues']
    >>> for key, value in sorted(feed['Schedule'].items()):
    ...     print('{:3} {}'.format(len(value), key))  # <3>
    ...
      1 conferences
    484 events
    357 speakers
     53 venues
    >>> feed['Schedule']['speakers'][-1]['name']  # <4>
    'Carina C. Zona'
    >>> feed['Schedule']['speakers'][-1]['serial']  # <5>
    141590
    >>> feed['Schedule']['events'][40]['name']
    'There *Will* Be Bugs'
    >>> feed['Schedule']['events'][40]['speakers']  # <6>
    [3471, 5199]


# END OSCONFEED_DEMO
"""

# BEGIN OSCONFEED
from urllib.request import urlopen
import warnings
import os
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'data/osconfeed.json'


def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg)  # <1>
        with urlopen(URL) as remote, open(JSON, 'wb') as local:  # 在with语句中使用两个上下文管理器（从Python 2.7和Python 3.1起允许这么做），分别用于读取和保存远程文件。
            local.write(remote.read())

    with open(JSON) as fp:
        return json.load(fp)  # <3>

# END OSCONFEED
feed = load()

### 19.1.1 使用动态属性访问JSON类数据

feed\['Schedule'\]\['events'\]\[40\]\['name'\]这种句法很冗长。在JavaScript中，可以使用feed.Schedule.events\[40\].name获取那个值。在Python中，可以实现一个近似字典的类（网上有大量实现）。

目标：实现一个FrozenJSON类，只支持读取，即只能访问数据。不过，这个类能递归，自动处理嵌套的映射和列表。

In [ ]:
# 目标：实现一个FrozenJSON类，只支持读取，即只能访问数据。不过，这个类能递归，自动处理嵌套的映射和列表。

from collections import abc
import keyword

class FrozenJSON:
    """
        一个只读接口，使用属性表示法访问JSON类对象
    """
    def __init__(self, mapping):
        # self.__data = dict(mapping) # 使用mapping参数构建一个字典。这么做有两个目的：(1)确保传入的是字典（或者是能转换成字典的对象）；(2)安全起见，创建一个副本。
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key): # 检查是否是python的关键字
                key += '_'
            self.__data[key] = value 

    def __getattr__(self, name): # 仅当没有指定名称（name）的属性时才调用__getattr__方法。
        if hasattr(self.__data, name):
            return getattr(self.__data, name) # 如果name是实例属性__data的属性，返回那个属性。调用keys等方法就是通过这种方式处理的。

        else:
            return FrozenJSON.build(self.__data[name]) # 否则，从self.__data中获取name键对应的元素，返回调用FrozenJSON.build（　）方法得到的结果。

    @classmethod
    def build(cls, obj): # 这是一个备选构造方法，@classmethod装饰器经常这么用。
        # Json的数据只有字典和列表两种类型
        if isinstance(obj, abc.Mapping): # 如果obj是映射，那就构建一个FrozenJSON对象。
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence): # 如果是MutableSequence对象，必然是列表，[插图]因此，我们把obj中的每个元素递归地传给.build（　）方法，构建一个列表。
            return [cls.build(item) for item in obj]
        else: # 如果既不是字典也不是列表，那么原封不动地返回元素。
            return obj
    

In [ ]:
# 检查 FrozenJSON 类
import json

filepath = 'chapter19/osconfeed.json'

with open(filepath, 'r') as fp:
    j = json.load(fp)

    feed = FrozenJSON(j)

    print(len(feed.Schedule.speakers))

    print(sorted(feed.Schedule.keys()))

    for key, value in sorted(feed.Schedule.items()):
        print('{:3} {}'.format(len(value), key))
    
    fp.close()

### 19.1.3 使用__new__方法以灵活的方式创建对象

其实，用于构建实例的是特殊方法__new__：这是个类方法（使用特殊方式处理，因此不必使用@classmethod装饰器），必须返回一个实例。返回的实例会作为第一个参数（即self）传给__init__方法。因为调用__init__方法时要传入实例，而且禁止返回任何值，所以__init__方法其实是“初始化方法”。真正的构造方法是__new__。

我们几乎不需要自己编写__new__方法，因为从object类继承的实现已经足够了。

骚操作的基础：即从__new__方法到__init__方法，是最常见的，但不是唯一的。__new__方法也可以返回其他类的实例，此时，解释器不会调用__init__方法。

In [ ]:
# 使用__new__方法取代build方法，构建可能是也可能不是FrozenJSON实例的新对象

from collections import abc
import keyword

class FrozenJSON2:
    """
        一个只读接口，使用属性表示法访问JSON类对象
    """
    def __new__(cls, obj): # __new__是类方法，第一个参数是类本身，余下的参数与__init__方法一样，只不过没有self。
        if isinstance(obj, abc.Mapping): 
            return super().__new__(cls)  #  默认的行为是委托给超类的__new__方法。这里调用的是object基类的__new__方法，把唯一的参数设为FrozenJSON。
        elif isinstance(obj, abc.MutableSequence):  # __new__方法中余下的代码与原先的build方法完全一样。
            return [cls.build(item) for item in obj] 
        else: 
            return obj

    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key): 
                key += '_'
            self.__data[key] = value 

    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name) 

        else:
            return FrozenJSON(self.__data[name]) # 之前，这里调用的是FrozenJSON.build方法，现在只需调用FrozenJSON构造方法。

    # 使用__new__内置函数代替：
    # @classmethod
    # def build(cls, obj): # 这是一个备选构造方法，@classmethod装饰器经常这么用。
    #     # Json的数据只有字典和列表两种类型
    #     if isinstance(obj, abc.Mapping): # 如果obj是映射，那就构建一个FrozenJSON对象。
    #         return cls(obj)
    #     elif isinstance(obj, abc.MutableSequence): # 如果是MutableSequence对象，必然是列表，[插图]因此，我们把obj中的每个元素递归地传给.build（　）方法，构建一个列表。
    #         return [cls.build(item) for item in obj]
    #     else: # 如果既不是字典也不是列表，那么原封不动地返回元素。
    #         return obj

In [ ]:
# 检查 FrozenJSON 类
import json

filepath = 'chapter19/osconfeed.json'

with open(filepath, 'r') as fp:
    j = json.load(fp)

    feed = FrozenJSON2(j)

    print(len(feed.Schedule.speakers))

    print(sorted(feed.Schedule.keys()))

    for key, value in sorted(feed.Schedule.items()):
        print('{:3} {}'.format(len(value), key))

    fp.close()

### 19.1.4 使用shelve模块调整OSCON数据源的结构

接下来的任务是，调整数据结构，以便自动获取所链接的记录。

shelve.open高阶函数返回一个shelve.Shelf实例，这是简单的键值对象数据库，背后由dbm模块支持，具有下述特点。
- shelve.Shelf是abc.MutableMapping的子类，因此提供了处理映射类型的重要方法。
- 此外，shelve.Shelf类还提供了几个管理I/O的方法，如sync和close；
- 它也是一个上下文管理器。
- 只要把新值赋予键，就会保存键和值。
- 键必须是字符串。
- 值必须是pickle模块能处理的对象。



In [ ]:
"""
schedule1.py: traversing OSCON schedule data
"""
# BEGIN SCHEDULE1
import warnings
import json

DB_NAME = 'data/schedule1_db'
CONFERENCE = 'conference.115'


def load_osconfeed():
    with open('chapter19/osconfeed.json', 'r') as fp:
        return json.load(fp)


class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)  #  这是使用关键字参数传入的属性构建实例的常用简便方式（详情参见下文）。
        # Record.__init__方法展示了一个流行的Python技巧。我们知道，对象的__dict__属性中存储着对象的属性--前提是类中没有声明__slots__属性，如9.8节所述。
        # 因此，更新实例的__dict__属性，把值设为一个映射，能快速地在那个实例中创建一堆属性。另外要处理不能作为属性名的健。

'''
双星号（**）：**kwargs  将参数以字典的形式导入
单星号（*）：*agrs  将所以参数以元组(tuple)的形式导入
'''


def load_db(db):
    raw_data = load_osconfeed()  # 如果本地没有副本，从网上下载JSON数据源。
    warnings.warn('loading ' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():  # 迭代集合（例如'conferences'、'events'，等等）。
        record_type = collection[:-1]  # record_type的值是去掉尾部's'后的集合名（即把'events'变成'event'）。
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])  # 使用record_type和'serial'字段构成key。
            record['serial'] = key  # 把'serial'字段的值设为完整的键。
            db[key] = Record(**record)  #  构建Record实例，存储在数据库中的key键名下。

# END SCHEDULE1

import shelve 

with shelve.open(DB_NAME) as db: # shelve.open函数打开现有的数据库文件，或者新建一个。
    if CONFERENCE not in db: # 判断数据库是否填充的简便方法是，检查某个已知的键是否存在；这里检查的键是conference.115，即conference记录（只有一个）的键。
        load_db(db) # 如果数据库是空的，那就调用load_db(db)，加载数据。

    speaker = db['speaker.3471'] # 获取一条speaker记录。
    print(type(speaker)) # 它是示例19-9中定义的Record类的实例。 <class '__main__.Record'>
    print(speaker.name) # 各个Record实例都有一系列自定义的属性，对应于底层JSON记录里的字段。  Anna Martelli Ravenscroft
    print(speaker.twitter) 
    db.close() # 一定要记得关闭shelve.Shelf对象。如果可以，使用with块确保Shelf对象会关闭。

### 19.1.5 使用特性获取链接的记录

@property装饰器装饰的方法为特性。

目标是，对于从Shelf对象中获取的event记录来说，读取它的venue或speakers属性时返回的不是编号，而是完整的记录对象。

**Record**

- __init__方法与schedule1.py脚本（见示例19-9）中的一样；为了辅助测试，增加了__eq__方法。

**DbRecord**

- Record类的子类，添加了__db类属性，用于设置和获取__db属性的set_db和get_db静态方法，用于从数据库中获取记录的fetch类方法，以及辅助调试和测试的__repr__实例方法。
- DbRecord.__db类属性的作用是存储打开的shelve.Shelf数据库引用，以便在需要使用数据库的DbRecord.fetch方法及Event.venue和Event.speakers属性中使用。

**Event**

- DbRecord类的子类，添加了用于获取所链接记录的venue和speakers属性，以及特殊的__repr__方法。

![Record Class UML](chapter19/RecordClassUML.png)

In [ ]:
import warnings
import inspect # inspect模块在load_db函数中使用
import json

DB_NAME = 'data/schedule2_db' # 因为要存储几个不同类的实例，所以我们要创建并使用不同的数据库文件；这里不用示例19-9中的'schedule1_db'，而是使用'schedule2_db'。
CONFERENCE = 'conference.115'


def load_osconfeed():
    with open('chapter19/osconfeed.json', 'r') as fp:
        return json.load(fp)


def load_db(db):
    raw_data = load_osconfeed()  # 如果本地没有副本，从网上下载JSON数据源。
    warnings.warn('loading ' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():  # 迭代集合（例如'conferences'、'events'，等等）。
        record_type = collection[:-1]  # record_type的值是去掉尾部's'后的集合名（即把'events'变成'event'）。
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])  # 使用record_type和'serial'字段构成key。
            record['serial'] = key  # 把'serial'字段的值设为完整的键。
            db[key] = Record(**record)  #  构建Record实例，存储在数据库中的key键名下。


class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
    def __eq__(self, other): # __eq__方法对测试有重大帮助。
        if isinstance(other, Record):
            return self.__dict__ == other.__dict__
        else:
            return NotImplemented

# 一个自定义的异常类型
class MissingDatabaseError(RuntimeError):
    """需要数据库，但没有指定数据库时抛出""" 
    # 自定义的异常通常是标志类，没有定义体。写一个文档字符串，说明异常的用途，比只写一个pass语句要好。

 
class DbRecord(Record): # DbRecord类扩展Record类
    __db = None #  __db类属性存储一个打开的shelve.Shelf数据库引用。

    @staticmethod # set_db是静态方法，以此强调不管调用多少次，效果始终一样。
    def set_db(db):
        DbRecord.__db = db # 即使调用Event.set_db(my_db)，__db属性仍在DbRecord类中设置。

    @staticmethod
    def get_db():
        return DbRecord.__db

    @classmethod # fetch是类方法，因此在子类中易于定制它的行为。
    def fetch(cls, ident):
        db = cls.get_db()
        try:
            return db[ident] # 从数据库中获取ident键对应的记录。
        except TypeError:
            if db is None: # 如果捕获到TypeError异常，而且db变量的值是None，抛出自定义的异常，说明必须设置数据库。
                msg = "database not set; call '{}.set_db(my_db)'"
                raise MissingDatabaseError(msg.format(cls.__name__))
            else: # 否则，重新抛出TypeError异常，因为我们不知道怎么处理。
                raise
    
    def __repr__(self):
        if hasattr(self, 'serial'): # 如果记录有serial属性，在字符串表示形式中使用。
            cls_name = self.__class__.__name__
            return '<{} serial={!r}'.format(cls_name, self.serial)
        else:
            return super().__repr__() # 否则，调用继承的__repr__方法。


# 重要的Event类
class Event(DbRecord): # Event类扩展DbRecord类
    @property
    def venue(self):
        key = 'venue.{}'.format(self.venue_serial) 
        return self.__class__.fetch(key) # 在venue特性中使用venue_serial属性构建key，然后传给继承自DbRecord类的fetch类方法（详情参见下文）。

    @property
    def speaker(self):
        if not hasattr(self, '_speaker_obs'): # speakers特性检查记录是否有_speaker_objs属性
            spkr_serials = self.__dict__['speakers'] # 如果没有，直接从__dict__实例属性中获取'speakers'属性的值，防止无限递归，因为这个特性的公开名称也是speakers。
            fetch = self.__class__.fatch # 获取fetch类方法的引用（稍后会说明这么做的原因）。
            self._speaker_objs = [  fetch('speaker.{}'.format(key))  # 使用fetch获取speaker记录列表，然后赋值给self._speaker_objs。
                                    for key in spkr_serials]
        return self._speaker_objs # 返回前面获取的列表。

    def __repr__(self):
        if hasattr(self, 'name'): # 如果记录有name属性，在字符串表示形式中使用。
            cls_name = self.__class__.__name__
            return '<{} {!r}>'.format(cls_name, self.name)
        else:
            return super().__repr__() # 否则，调用继承的__repr__方法。


# --------------------------------------------- RUN ----------------------------------------------------
with shelve.open(DB_NAME) as db: # shelve.open函数打开现有的数据库文件，或者新建一个。
    if CONFERENCE not in db: # 判断数据库是否填充的简便方法是，检查某个已知的键是否存在；这里检查的键是conference.115，即conference记录（只有一个）的键。
        load_db(db) # 如果数据库是空的，那就调用load_db(db)，加载数据。

    DbRecord.set_db(db)

    event = DbRecord.fetch('event.33950')
    #print(event.venue)
    print(event)


    db.close()


为什么不直接使用self.fetch(key)呢？

哪怕只有一个事件记录有名为'fetch'的键，那么在那个Event实例中，self.fetch获取的是fetch字段的值，而不是Event继承自DbRecord的fetch类方法。这个缺陷不明显，很容易被测试忽略；

In [ ]:
import warnings
import inspect # inspect模块在load_db函数中使用
import json

DB_NAME = 'data/schedule2_db' # 因为要存储几个不同类的实例，所以我们要创建并使用不同的数据库文件；这里不用示例19-9中的'schedule1_db'，而是使用'schedule2_db'。
CONFERENCE = 'conference.115'


def load_osconfeed():
    with open('chapter19/osconfeed.json', 'r') as fp:
        return json.load(fp)


def load_db(db):
    raw_data = load_osconfeed()  # 如果本地没有副本，从网上下载JSON数据源。
    warnings.warn('loading ' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():  # 迭代集合（例如'conferences'、'events'，等等）。
        record_type = collection[:-1]  # record_type的值是去掉尾部's'后的集合名（即把'events'变成'event'）。
        
        cls_name = record_type.capitalize() #  把record_type变量的值首字母变成大写（例如，把'event'变成'Event'），获取可能的类名。
        cls = globals().get(cls_name, DbRecord) # 从模块的全局作用域中获取那个名称对应的对象；如果找不到对象，使用DbRecord
        if inspect.isclass(cls) and issubclass(cls, DbRecord): # 如果获取的对象是类，而且是DbRecord的子类……
            factory = cls # ……把对象赋值给factory变量。因此，factory的值可能是DbRecord的任何一个子类，具体的类取决于record_type的值。
        else:
            factory = DbRecord # 否则，把DbRecord赋值给factory变量。

        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])  
            record['serial'] = key  
            db[key] = factory(**record)  #  ……存储在数据库中的对象由factory构建，factory可能是DbRecord类，也可能是根据record_type的值确定的某个子类。


class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
    def __eq__(self, other): # __eq__方法对测试有重大帮助。
        if isinstance(other, Record):
            return self.__dict__ == other.__dict__
        else:
            return NotImplemented

# 一个自定义的异常类型
class MissingDatabaseError(RuntimeError):
    """需要数据库，但没有指定数据库时抛出""" 
    # 自定义的异常通常是标志类，没有定义体。写一个文档字符串，说明异常的用途，比只写一个pass语句要好。

 
class DbRecord(Record): # DbRecord类扩展Record类
    __db = None #  __db类属性存储一个打开的shelve.Shelf数据库引用。

    @staticmethod # set_db是静态方法，以此强调不管调用多少次，效果始终一样。
    def set_db(db):
        DbRecord.__db = db # 即使调用Event.set_db(my_db)，__db属性仍在DbRecord类中设置。

    @staticmethod
    def get_db():
        return DbRecord.__db

    @classmethod # fetch是类方法，因此在子类中易于定制它的行为。
    def fetch(cls, ident):
        db = cls.get_db()
        try:
            return db[ident] # 从数据库中获取ident键对应的记录。
        except TypeError:
            if db is None: # 如果捕获到TypeError异常，而且db变量的值是None，抛出自定义的异常，说明必须设置数据库。
                msg = "database not set; call '{}.set_db(my_db)'"
                raise MissingDatabaseError(msg.format(cls.__name__))
            else: # 否则，重新抛出TypeError异常，因为我们不知道怎么处理。
                raise
    
    def __repr__(self):
        if hasattr(self, 'serial'): # 如果记录有serial属性，在字符串表示形式中使用。
            cls_name = self.__class__.__name__
            return '<{} serial={!r}'.format(cls_name, self.serial)
        else:
            return super().__repr__() # 否则，调用继承的__repr__方法。


# 重要的Event类
class Event(DbRecord): # Event类扩展DbRecord类
    @property
    def venue(self):
        key = 'venue.{}'.format(self.venue_serial) 
        return self.__class__.fetch(key) # 在venue特性中使用venue_serial属性构建key，然后传给继承自DbRecord类的fetch类方法（详情参见下文）。

    @property
    def speakers(self):
        if not hasattr(self, '_speaker_obs'): # speakers特性检查记录是否有_speaker_objs属性
            spkr_serials = self.__dict__['speakers'] # 如果没有，直接从__dict__实例属性中获取'speakers'属性的值，防止无限递归，因为这个特性的公开名称也是speakers。
            fetch = self.__class__.fetch # 获取fetch类方法的引用（稍后会说明这么做的原因）。
            self._speaker_objs = [  fetch('speaker.{}'.format(key))  # 使用fetch获取speaker记录列表，然后赋值给self._speaker_objs。
                                    for key in spkr_serials]
        return self._speaker_objs # 返回前面获取的列表。

    def __repr__(self):
        if hasattr(self, 'name'): # 如果记录有name属性，在字符串表示形式中使用。
            cls_name = self.__class__.__name__
            return '<{} {!r}>'.format(cls_name, self.name)
        else:
            return super().__repr__() # 否则，调用继承的__repr__方法。


# --------------------------------------------- RUN ----------------------------------------------------
with shelve.open(DB_NAME) as db: # shelve.open函数打开现有的数据库文件，或者新建一个。
    if CONFERENCE not in db: # 判断数据库是否填充的简便方法是，检查某个已知的键是否存在；这里检查的键是conference.115，即conference记录（只有一个）的键。
        load_db(db) # 如果数据库是空的，那就调用load_db(db)，加载数据。

    DbRecord.set_db(db)

    event = DbRecord.fetch('event.33950')
    print(event.venue.name)

    for spkr in event.speakers:
        print('{0.serial} : {0.name}'.format(spkr))

    db.close()


_getattr__方法、hasattr函数、getattr函数、@property装饰器和__dict__属性，来实现动态属性。

特性经常用于把公开的属性变成使用读值方法和设值方法管理的属性，且在不影响客户端代码的前提下实施业务规则

## 19.2 使用特性验证属性

前面只介绍了如何使用@property装饰器实现只读特性。本节要创建一个可读写的特性。



In [ ]:
# 已经发布的第一代代码：

class LineItem:
    def __init__(self, description, weight, price):
        self.__description = description
        self.__weight = weight
        self.__price = price

    def subtotal(self):
        return self.__weight * self.__price

In [ ]:
# 发现发布的代码有问题，比方weight可以为负。需要修正，但是为了不破话其他代码的引用，我们采用特性来打补丁。

class LineItem:
    def __init__(self, description, weight, price):
        self.__description = description
        self.__weight = weight
        self.__price = price

    def subtotal(self):
        return self.__weight * self.__price

    # 使用特性打补丁
    @property
    def weight(self): # 实现特性的方法，其名称都与公开属性的名称一样——weight。
        return self.__weight

    @weight.setter
    def weight(self, value):
        if value > 0 :
            self.__weight = value
        else:
            raise ValueError('value must be > 0')


In [ ]:
# 测试

raisins = LineItem('Golden raisins', 10, 6.95)
print(raisins.subtotal())

raisins.weight = -10.0

print(raisins.subtotal())

上面虽然修复了重量weight这个属性不能为负，但是价格还是可以为负的。我们当然可以将价格也做成特性，但是这是和weight重复的代码，只是换了个名字而已，因此代码失去了优雅。而去除重复的方法是抽象。

抽象特性的定义有两种方式：
- 特性工厂函数
- 描述符类（更灵活，下章介绍）

## 19.3 特性全解析

虽然内置的property经常用作装饰器，但它其实是一个类。在Python中，函数和类通常可以互换，因为二者都是可调用的对象，而且没有实例化对象的new运算符，所以调用构造方法与调用工厂函数没有区别。此外，只要能返回新的可调用对象，代替被装饰的函数，二者都可以用作装饰器。

In [ ]:
# property的经典用法，即使用了这个类的构造函数

class LineItem:
    def __init__(self, description, weight, price):
        self.__description = description
        self.__weight = weight
        self.__price = price

    def subtotal(self):
        return self.__weight * self.__price


    def get_weight(self):
        return self.__weight

    def set_weight(self, value):
        if value > 0 :
            self.__weight = value
        else:
            raise ValueError('value must be > 0')

    weight = property(get_weight, set_weight)

    # 某些情况下，这种经典形式比装饰器句法好；稍后讨论的特性工厂函数就是一例。

### 19.3.1 特性会覆盖实例属性

特性都是类属性，但是特性管理的其实是实例属性的存取。

如果实例和所属的类有同名数据属性，那么实例属性会覆盖（或称遮盖）类属性

本节的主要观点是，obj.attr这样的表达式不会从obj开始寻找attr，而是从obj.__class__开始，而且，仅当类中没有名为attr的特性时，Python才会在obj实例中寻找。这条规则不仅适用于特性，还适用于一整类描述符——覆盖型描述符（overridingdescriptor）。第20章会进一步讨论描述符，那时你会发现，特性其实是覆盖型描述符。

In [24]:
class Class: # 这个类有两个类属性：data数据属性和prop特性。
    data = 'the class data attr'

    @property
    def prop(self):
        return 'the prop value'

obj = Class()
print(vars(obj)) # vars函数返回obj的__dict__属性，表明没有实例属性。
print(obj.data)
obj.data = 'baar' # 为obj.data赋值，创建一个实例属性。
print(vars(obj))
print(obj.data) # 现在读取obj.data，获取的是实例属性的值。从obj实例中读取属性时，实例属性data会遮盖类属性data。
print(Class.data) # Class.data属性的值完好无损。

# #实例属性不会遮盖类特性#
print('#实例属性不会遮盖类特性#')

print(Class.prop) # 直接从Class中读取prop特性，获取的是特性对象本身，不会运行特性的读值方法。

print(obj.prop) # 读取obj.prop会执行特性的读值方法

# obj.prop = 'foo'  # 因为特性没有设置权限，因此会返回 AtributeError: can't set attribute
obj.__dict__['prop'] = 'foo' # 但是可以直接把'prop'存入obj.__dict__。
print(vars(obj)) # 可以看到，obj现在有两个实例属性：data和prop。

print(obj.prop) # 然而，读取obj.prop时仍会运行特性的读值方法。特性没被实例属性遮盖。

Class.prop = 'baz' # 覆盖Class.prop特性，销毁特性对象。原本是<property object>对象，现在只是个字符串了

print(obj.prop) # 现在，obj.prop获取的是实例属性。Class.prop不是特性了，因此不会再覆盖obj.prop。

# #新添的类特性遮盖现有的实例属性#
print("#新添的类特性遮盖现有的实例属性#")

print(obj.data) # obj.data获取的是实例属性data。

print(Class.data) # Class.data获取的是类属性data。

Class.data = property(lambda self: 'the "data" prop value') # 使用新特性覆盖Class.data。

print(obj.data) # 现在，obj.data被Class.data特性遮盖了。

del Class.data #  删除特性。

print(obj.data) # 现在恢复原样，obj.data获取的是实例属性data。

{}
the class data attr
{'data': 'baar'}
baar
the class data attr
#实例属性不会遮盖类特性#
the prop value
{'data': 'baar', 'prop': 'foo'}
the prop value
foo
#新添的类特性遮盖现有的实例属性#
baar
the class data attr
the "data" prop value
baar


### 19.3.2 特性文档

控制台中的help（　）函数或IDE等工具需要显示特性的文档时，会从特性的__doc__属性中提取信息。

为特性写文档有两种方式：


In [ ]:
# 方式一：

weight = property(get_weight, set_weight, doc='weight in kilograms')

# 方法二：
@property
def bar(self):
    '''The bar attribute'''
    return self.__dict__['bar']

@bar.setter
def bar(self, value):
    self.__dict__['bar'] = value